# Lab: Preprocess Data
## Purpose:
- Apply pre-processing techniques to clean data
- Removing/replacing markup tags
- Using Unicode categories

### Topics:
- Removing markup tags
- Unicode characters

### Steps
* Implement a function to strip HTML tags from raw text.
* Extend the function to clean common HTML entities like `&lt;`, `&gt;`, and `&amp;`.
* Use the `unicodedata` package to explore character categories.
* Adapt a cleaning function to keep letters, numbers, punctuation, and whitespace while removing emojis and other symbols.

Date: 2026-02-20

Source: https://colab.research.google.com/github/google-deepmind/ai-foundations/blob/master/course_2/gdm_lab_2_1_preprocess_data.ipynb

References: https://github.com/google-deepmind/ai-foundations
- GDM GH repo used in AI training courses at the university & college level.